In [57]:
!python main.py play --agents q_agent --train 1 --scenario coin-heaven --n-rounds 10

size of dictionary: 70
 10%|████▍                                       | 1/10 [00:07<01:06,  7.35s/it]/home/vivi/uni/bomberman/agent_code/q_agent/callbacks.py:311: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if (neighbours[j] == game_state['bombs'][0]).all():
100%|███████████████████████████████████████████| 10/10 [00:35<00:00,  3.52s/it]


In [59]:
!python main.py play --agents q_agent q_agent random_agent --scenario classic --n-rounds 4

size of dictionary: 70
nonzero rows: 1050
size of dictionary: 70
nonzero rows: 1050
  0%|                                                     | 0/4 [00:00<?, ?it/s]/home/vivi/uni/bomberman/agent_code/q_agent/callbacks.py:341: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if any([(opponent_pos[0] == my_neighbours[0] and opponent_pos[1] == my_neighbours[1]) for neighbour in neighbours]): # opponent is neighbour of agent
/home/vivi/uni/bomberman/agent_code/q_agent/callbacks.py:307: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if (neighbours[j] == game_state['bombs'][0]).all():
100%|█████████████████████████████████████████████| 4/4 [00:24<00:00,  6.15s/it]


In [56]:
!python main.py play --agents q_agent --scenario coin-heaven --n-rounds 3

size of dictionary: 70
nonzero rows: 19
 33%|███████████████                              | 1/3 [00:06<00:13,  6.88s/it]/home/vivi/uni/bomberman/agent_code/q_agent/callbacks.py:310: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if (neighbours[j] == game_state['bombs'][0]).all():
100%|█████████████████████████████████████████████| 3/3 [00:09<00:00,  3.23s/it]


In [193]:
!python main.py play --agents q_agent random_agent rule_based_agent --scenario coin-heaven --n-rounds 2

size of dictionary: 70
nonzero rows: 35
  0%|                                                     | 0/2 [00:00<?, ?it/s]/home/vivi/uni/bomberman/agent_code/q_agent/callbacks.py:339: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if any([(opponent_pos[0] == my_neighbours[0] and opponent_pos[1] == my_neighbours[1]) for neighbour in neighbours]): # opponent is neighbour of agent
/home/vivi/uni/bomberman/agent_code/q_agent/callbacks.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if (neighbours[j] == game_state['bombs'][0]).all():
100%|█████████████████████████████████████████████| 2/2 [00:15<00:00,  7.55s/it]


In [194]:
!python main.py play --agents q_agent --scenario clean --n-rounds 3

size of dictionary: 70
nonzero rows: 35
  0%|                                                     | 0/3 [00:04<?, ?it/s]


In [ ]:
!python main.py play --my_agent q_agent --scenario classic --save-replay 

In [ ]:
!python main.py replay "replays/replay.pt" --turn-based

In [ ]:
reward = np.loadtxt("agent_code/q_agent/rewards.txt",dtype = float)
plt.plot(reward,'.')
plt.xlabel("rounds")
plt.ylabel("reward")
#plt.ylim(-100,100)

In [3]:
import numpy as np
import itertools

In [15]:
dic = {}
ACTIONS = ['UP', 'RIGHT', 'DOWN', 'LEFT', 'WAIT', 'BOMB']

NEIGHBOURING_FIELDS = [-1,0,1,2] # the possible values for neighbouring fields
GAME_MODE = [0,1,2] # the possible values for game mode
CURRENT_FIELD = [0,1,2,3,4] # the possible values for current filed

# rotate clockwise
def get_arrangements(array):
    #return np.unique([ np.roll(array, -i) for i in range(0,4) ], axis=0)
    return [ np.roll(array, -i) for i in range(0,4) ]
    
# get all possible combinations
def get_variations(array, r=4):
    #return np.unique([ item for item in list(itertools.product(array, repeat=r)) ], axis=0)
    return [ item for item in list(itertools.product(array, repeat=r)) ]

# compute state to index for the Q-table
def build_state_to_index(arr1 = NEIGHBOURING_FIELDS, arr2 = GAME_MODE, arr3 = CURRENT_FIELD):
    i = 0
    perm = get_variations(arr1)
    comb = np.array(np.meshgrid(arr2, arr3)).T.reshape(-1,2) # combinations of arr2 and arr3
    print(comb, len(comb))
    step = len(comb)
    for p in perm:
        already_there = []
        arrangements = get_arrangements(p)
        for a in arrangements:
            if tuple(a) in dic.keys():
                already_there.append(True)
            else:
                already_there.append(False)
        if sum(already_there) == 0: # p not in dictionary
            value = [v for v in range(i, i+step)] # value indices for p
            i += step # update i
            dic.update({tuple(p) : value}) # add in dictionary
            
def get_state_index(state):
    arrangements = get_arrangements(state[-4:]) # rotations of neighbouring fields
    mode_and_field = state[:2]
    value_index = 0
    comb = np.array(np.meshgrid(GAME_MODE, CURRENT_FIELD)).T.reshape(-1,2)
    for i, c in enumerate(comb):
        if (c == mode_and_field).all():
            value_index = i
            break
    for j, a in enumerate(arrangements):
        if tuple(a) in dic.keys():
            #print("state: ", state, "index", dic[tuple(a)][value_index])
            return dic[tuple(a)][value_index], j
        else:
            continue

                
build_state_to_index()

print(f"size of dictionary: {len(dic)}")

print(dic.keys())

#print(dic[tuple([2,  2, -1, -1])])
#print(dic[tuple([2 ,-1 ,-1 , 2])])

print(get_arrangements([2,  2, -1, -1]))
#print(get_arrangements([1, -1, -1 ,1]))
#print(get_state_index([ 2,  3,  1, -1, -1 , 1]))
print(len(get_variations(NEIGHBOURING_FIELDS)))

[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [0 4]
 [1 0]
 [1 1]
 [1 2]
 [1 3]
 [1 4]
 [2 0]
 [2 1]
 [2 2]
 [2 3]
 [2 4]] 15
size of dictionary: 70
dict_keys([(-1, -1, -1, -1), (-1, -1, -1, 0), (-1, -1, -1, 1), (-1, -1, -1, 2), (-1, -1, 0, 0), (-1, -1, 0, 1), (-1, -1, 0, 2), (-1, -1, 1, 0), (-1, -1, 1, 1), (-1, -1, 1, 2), (-1, -1, 2, 0), (-1, -1, 2, 1), (-1, -1, 2, 2), (-1, 0, -1, 0), (-1, 0, -1, 1), (-1, 0, -1, 2), (-1, 0, 0, 0), (-1, 0, 0, 1), (-1, 0, 0, 2), (-1, 0, 1, 0), (-1, 0, 1, 1), (-1, 0, 1, 2), (-1, 0, 2, 0), (-1, 0, 2, 1), (-1, 0, 2, 2), (-1, 1, -1, 1), (-1, 1, -1, 2), (-1, 1, 0, 0), (-1, 1, 0, 1), (-1, 1, 0, 2), (-1, 1, 1, 0), (-1, 1, 1, 1), (-1, 1, 1, 2), (-1, 1, 2, 0), (-1, 1, 2, 1), (-1, 1, 2, 2), (-1, 2, -1, 2), (-1, 2, 0, 0), (-1, 2, 0, 1), (-1, 2, 0, 2), (-1, 2, 1, 0), (-1, 2, 1, 1), (-1, 2, 1, 2), (-1, 2, 2, 0), (-1, 2, 2, 1), (-1, 2, 2, 2), (0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 0, 2), (0, 0, 1, 1), (0, 0, 1, 2), (0, 0, 2, 1), (0, 0, 2, 2), (0, 1, 0, 1), (0, 1, 0, 2), (0, 1, 1, 1), (0,

In [23]:
model_trained = np.load("agent_code/q_agent/my-saved-model.npy")

In [24]:
state1 = [2,0,-1,-1,2,2]
state2 = [2,0,-1,2,2,-1]

i1, r1 = get_state_index(state1)
i2, r2 = get_state_index(state2)
print(i1, r1, i2, r2)

print(model_trained[i1])

190 0 190 3
[-77.14352453 -78.49698771 -61.17325969 -73.70228508 -49.98128351
 -58.89993109]


In [43]:
pos = (1,1)
sub = [(1,0), (0,-1), (-1,0), (0,1)] # left, down, right, up
neighbours = []
for i in sub:
    neighbour = np.subtract(pos, i)
    if (0 <= neighbour[0] < 17) and (0 <= neighbour[1] < 17): # check if in game borders
        neighbours.append(neighbour)
print(neighbours)

a = (17, 14)
print(np.array(a))
r = [[12, 14], [13 ,14], [14 ,14], [12, 15], [13, 15], [14, 15], [12, 16], [13, 16], [14, 16]]
print((len(list(filter (lambda x : (x == np.array(a)).all(), r))) > 0))

[array([0, 1]), array([1, 2]), array([2, 1]), array([1, 0])]
[17 14]
False


In [4]:
def get_region(my_pos, pos):
    if pos[0] == my_pos[0] and pos[1] > my_pos[1]: # up
        r1 = np.array([ [pos[0]-1, y] for y in range(pos[1], pos[1]+3) ])
        r2 = np.array([ [pos[0], y] for y in range(pos[1], pos[1]+3) ])
        r3 = np.array([ [pos[0]+1, y] for y in range(pos[1], pos[1]+3) ])         
        r12 = np.concatenate((r1, r2), axis = 0)
        return np.append(r12,r3).reshape(-1,2)
    elif pos[0] == my_pos[0] and pos[1] < my_pos[1]: # down
        r1 = np.array([ [pos[0]-1, y] for y in range(pos[1]-2, pos[1]+1) ])
        r2 = np.array([ [pos[0], y] for y in range(pos[1]-2, pos[1]+1) ])
        r3 = np.array([ [pos[0]+1, y] for y in range(pos[1]-2, pos[1]+1) ])
        r12 = np.concatenate((r1, r2), axis = 0)
        return np.append(r12,r3).reshape(-1,2)
    elif pos[1] == my_pos[1] and pos[0] > my_pos[0]: # right
        r1 = np.array([ [x, pos[1]+1] for x in range(pos[0], pos[0]+3) ])
        r2 = np.array([ [x, pos[1]] for x in range(pos[0], pos[0]+3) ])
        r3 = np.array([ [x, pos[1]-1] for x in range(pos[0], pos[0]+3) ])
        r12 = np.concatenate((r1, r2), axis = 0)
        return np.append(r12,r3).reshape(-1,2)
    else: # left
        r1 = np.array([ [x, pos[1]+1] for x in range(pos[0]-2, pos[0]+1) ])
        r2 = np.array([ [x, pos[1]] for x in range(pos[0]-2, pos[0]+1) ])
        r3 = np.array([ [x, pos[1]-1] for x in range(pos[0]-2, pos[0]+1) ])
        r12 = np.concatenate((r1, r2), axis = 0)
        return np.append(r12,r3).reshape(-1,2)
    
    
print(get_region((1,0),(1,1)))

[[0 1]
 [0 2]
 [0 3]
 [1 1]
 [1 2]
 [1 3]
 [2 1]
 [2 2]
 [2 3]]


In [116]:
a = [0,1]
b = [0,1,1,1]
#print(get_variations(a))
b_ = get_arrangements(b)
#for i in b_:
#    print(tuple(i))
#print(b[-3:])


pos = (15,15)    
def get_neighbours(pos): # check if in game border ? 
    sub = [(1,0), (0,1), (-1,0), (0,-1)] # left, down, right, up
    neighbours = []
    for i in sub:
        neighbour = np.subtract(pos, i)
        if (0 <= neighbour[0] < 17) and (0 <= neighbour[1] < 17):
            neighbours.append(neighbour)
    return neighbours

n = get_neighbours(pos)
print(n, n[1])

pos1 = [[15,15], [16,15]]

print(any([(pos[0] == i[0] and pos[1] == i[1]) for i in pos1]))

[array([14, 15]), array([15, 14]), array([16, 15]), array([15, 16])] [15 14]
True


In [ ]:
def check_for_wall(pos, field=game_state['field']):
        #if (position >= 17).any() or (position < 0).any():
        #    print("out of field")
        #    return True
        return field[pos[0],pos[1]] == -1 # wall

    # get vertical and horizontal region from the position
def get_vh_region(pos, n=3):
    horizontal = np.array([ [x, pos[1]] for x in range(pos[0]-n, pos[0]+(n+1)) ])
    vertical = np.array([ [pos[0], y] for y in range(pos[1]-n, pos[1]+(n+1)) ])
    return np.concatenate((vertical, horizontal), axis = 0)

In [9]:
model_trained = np.load("agent_code/q_agent/my-saved-model.npy")

In [ ]:

print(np.shape(model_trained))
print(f"# nonzero rows { sum(np.apply_along_axis(np.any, axis=1, arr=model_trained)) }")
print(model_trained[548])

state = [ 0, -1,  2,  2, -1,  2]
print(state)
index = -1
rotation = -1
temp = get_arrangements(state[1:5]) # rotations of neighbouring fields
mode_and_field = [ state[-1], state[0] ]
value_index = 0
comb = np.array(np.meshgrid(GAME_MODE, CURRENT_FIELD)).T.reshape(-1,2)
for i, c in enumerate(comb):
    if (c == mode_and_field).all():
        value_index = i
        print(value_index)
        break
rotation = 0
for i, t in enumerate(temp):
    if (t == state[1:5]).all():
        rotation = i
        print(rotation)
for j, a in enumerate(temp):
    if tuple(a) in dic.keys():
        #print("state: ", state, "index", dic[tuple(a)][value_index])
        index = dic[tuple(a)][value_index]
        print(index)
        break
        
action = np.argmax(model_trained[index]) # Exploit learned values
if action < 4 and rotation != 0: # move and rotated state
    action = (action + rotation) % 4
print(ACTIONS[action])

In [40]:
from datetime import datetime

now = datetime.now() 

year = now.strftime("%Y")
print("year:", year)

month = now.strftime("%m")
print("month:", month)

day = now.strftime("%d")
print("day:", day)

time = now.strftime("%H:%M:%S")
print("time:", time)

date_time = now.strftime("%H_%M_%S")
print("date and time:",date_time)

year: 2022
month: 03
day: 09
time: 14:56:48
date and time: 14_56_48


In [55]:
import time
start = time.time()


time.sleep(5)
end  = time.time()
print(end-start)

5.003371000289917
